In [8]:
import pandas as pd
import numpy as np
import joblib
import shap
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import os

# Configuration

In [17]:
MODEL_PATH = '/Users/singhanyadav/Documents/Python/UMAMI /catboost_model.joblib'  # Should be latest trained model
SCHEMA_PATH = '/Users/singhanyadav/Documents/Python/UMAMI /model_schema.joblib'   # To check Base Features in Dataset
NEW_DATA_PATH = '/Users/singhanyadav/Documents/Python/UMAMI /ML_data_media.csv'  # Base CSV file
TARGET_COLUMN = "Total Cell count (in 0.33cm2)"

# Testing with Different Dataset with Variations

In [6]:
new_df = pd.read_csv("ML_data_augmented.csv")
X_new = new_df.drop(columns=[TARGET_COLUMN])
y_new = new_df[TARGET_COLUMN]

# Running a check via Model

In [19]:
model_exists = os.path.exists(MODEL_PATH) and os.path.exists(SCHEMA_PATH)

retrain_needed = True
if model_exists:
    old_model = joblib.load(MODEL_PATH)
    old_schema = joblib.load(SCHEMA_PATH)
    current_schema = X_new.columns.tolist()

    if set(old_schema) == set(current_schema):
        print(" Schema unchanged. Checking model performance...")
        y_pred = old_model.predict(X_new)
        r2 = r2_score(y_new, y_pred)
        print("R^2 on new data:", r2)

        if r2 >= R2_THRESHOLD:
            retrain_needed = False
            print(" R^2 above threshold. Skipping retraining.")
        else:
            print(f" R^2 below threshold ({R2_THRESHOLD}). Retraining required.")
    else:
        print(" Schema changed. Retraining required.")

 Schema changed. Retraining required.


# Retrain Model if Needed

In [21]:
if retrain_needed:
    print(" Retraining CatBoost model...")
    X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

    model = CatBoostRegressor(verbose=0)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    print(" Evaluation Metrics:")
    print("MAE:", mean_absolute_error(y_test, y_pred))
    print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
    print("R^2:", r2_score(y_test, y_pred))

 Retraining CatBoost model...
 Evaluation Metrics:
MAE: 1315.0832905378059
RMSE: 1605.3769828664372
R^2: 0.722305049394022


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
# Saving new model
    joblib.dump(model, newcatboost_model.joblib)
    joblib.dump(X_new.columns.tolist(), newmodel_schema)
    print(" Model and schema saved.")
else:
    print(" Using existing model. No retraining performed.")